In [ ]:
import win32com.client as win32

# 한글 프로그램 열기
hwp = win32.gencache.EnsureDispatch("HWPFrame.HwpObject")
hwp.Open(r"C:\Jimin\cg-suri-hwp2txt\test\raw_data\(공고 제2024-881호) Burr Saw등 절삭기류 치료재료 비용 산정 가능 행위 목록 공고(고시 2024-258호,12.18. 관련).hwpx")  # hwp 파일 경로로 변경

# 전체 텍스트 추출
text = hwp.GetTextFile("Text")
print("본문 전체 텍스트:\n", text)

# 표 추출 예시: 표를 순회하며 셀 내용 출력
# (공문은 표가 없는 경우도 많으니, 표 위치를 자동 탐색)
def extract_table_contents(hwp):
    tables = []
    ctrl = hwp.HeadCtrl
    while ctrl is not None:
        if ctrl.CtrlID == 28:  # 28 : 표(Table) 컨트롤
            table = []
            nRow = ctrl.RowCount
            nCol = ctrl.ColCount
            for r in range(nRow):
                row = []
                for c in range(nCol):
                    try:
                        cell = ctrl.Cell(r, c)
                        row.append(cell.GetText())
                    except Exception:
                        row.append("")
                table.append(row)
            tables.append(table)
        ctrl = ctrl.Next
    return tables

tables = extract_table_contents(hwp)
for idx, table in enumerate(tables, 1):
    print(f"\n[표 {idx}]")
    for row in table:
        print('\t'.join(row))

hwp.Quit()


In [4]:
import os

# PDFConverter는 이미 작성하신 클래스에서 import한다고 가정합니다.
from converter import PDFConverter  

def convert_all_office_files_in_folder(input_folder: str, output_folder: str = None):
    """
    지정한 폴더 내의 모든 DOCX, HWP, HWPX 파일을 PDF로 변환합니다.

    Args:
        input_folder (str): 입력 폴더 경로
        output_folder (str, optional): 출력 폴더 경로 (기본값: 입력 폴더와 동일)

    Returns:
        list of str: 변환된 PDF 파일 경로 리스트
    """
    converter = PDFConverter()
    output_folder = output_folder or input_folder
    os.makedirs(output_folder, exist_ok=True)

    converted_files = []

    for filename in os.listdir(input_folder):
        filepath = os.path.join(input_folder, filename)
        if not os.path.isfile(filepath):
            continue

        file_ext = os.path.splitext(filename.lower())[1]
        if file_ext in ['.hwp', '.hwpx', '.docx', '.doc']:
            with open(filepath, 'rb') as f:
                file_bytes = f.read()
            
            try:
                pdf_bytes = converter._convert_office_to_pdf_bytes(file_bytes, file_ext)
                pdf_filename = os.path.splitext(filename)[0] + ".pdf"
                pdf_output_path = os.path.join(output_folder, pdf_filename)

                with open(pdf_output_path, 'wb') as f_out:
                    f_out.write(pdf_bytes)

                print(f"✅ 변환 완료: {filename} → {pdf_filename}")
                converted_files.append(pdf_output_path)

            except Exception as e:
                print(f"❌ 변환 실패: {filename} → {str(e)}")

    return converted_files

# 예시 실행
if __name__ == "__main__":
    input_dir = "./raw_data"
    output_dir = "./"
    convert_all_office_files_in_folder(input_dir, output_dir)


임시 파일 삭제: C:\Users\sokch\AppData\Local\Temp\tmpg_4cb7zw.hwpx
❌ 변환 실패: (공고 제2024-881호) Burr Saw등 절삭기류 치료재료 비용 산정 가능 행위 목록 공고(고시 2024-258호,12.18. 관련).hwpx → 파일 변환 실패: HWP PDF 변환 실패: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)
임시 파일 삭제: C:\Users\sokch\AppData\Local\Temp\tmpzq0512fl.hwpx
❌ 변환 실패: (제2025-23호) 요양급여의 적용기준 및 방법에 관한 세부사항 일부개정.hwpx → 파일 변환 실패: HWP PDF 변환 실패: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)
임시 파일 삭제: C:\Users\sokch\AppData\Local\Temp\tmp1byfk044.hwp
❌ 변환 실패: [별표 7] 진료코드(제20조제7항 관련)(자동차보험진료수가 심사업무처리에 관한 규정) (1).hwp → 파일 변환 실패: HWP PDF 변환 실패: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)
임시 파일 삭제: C:\Users\sokch\AppData\Local\Temp\tmpu0w3_6f7.hwpx
❌ 변환 실패: [의과 1기관 4사례] 추간판 장애 등 상병으로 통증조절 위해 입원한 사례 (척추1A).hwpx → 파일 변환 실패: HWP PDF 변환 실패: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)
임시 파일 삭제: C:\Users\sokch\AppData\Local\Temp\tmptw92pm69.hwp
❌ 변환 실패: 고시 제2025-102호 [일반원칙]항진균제.hwp → 파일 변환 실패: HWP PDF 변환 실패: (-2147221005, '잘못된 클래스 문자열입니다.', None, None)


In [5]:
import os
import subprocess
import shutil
import csv

INPUT_FOLDER = "./raw_data"
OUTPUT_FOLDER = "./output"
UNSUPPORTED_CSV = os.path.join(OUTPUT_FOLDER, "미처리_HWP_파일목록.csv")

def convert_hwpx_to_pdf(input_path, output_path):
    try:
        subprocess.run([
            "soffice", "--headless", "--convert-to", "pdf", input_path,
            "--outdir", output_path
        ], check=True)
        print(f"✅ 변환 완료: {os.path.basename(input_path)}")
        return True
    except Exception as e:
        print(f"❌ 변환 실패: {input_path} → {e}")
        return False

def ensure_output_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def convert_all_documents(input_folder, output_folder):
    ensure_output_folder(output_folder)

    unsupported_list = []
    converted_count = 0

    for filename in os.listdir(input_folder):
        filepath = os.path.join(input_folder, filename)
        ext = os.path.splitext(filename.lower())[1]
        if ext not in ['.hwp', '.hwpx']:
            continue

        if ext == ".hwpx":
            success = convert_hwpx_to_pdf(filepath, output_folder)
            if success:
                converted_count += 1
        elif ext == ".hwp":
            print(f"⚠ HWP 변환 불가, 기록: {filename}")
            unsupported_list.append([filename, filepath])

    # 기록
    if unsupported_list:
        with open(UNSUPPORTED_CSV, 'w', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(["파일명", "경로"])
            writer.writerows(unsupported_list)
        print(f"\n📄 변환 불가 파일 기록됨: {UNSUPPORTED_CSV}")

    print(f"\n✨ 총 변환 완료: {converted_count}개")
    print(f"📌 변환 불가(HWP): {len(unsupported_list)}개")

if __name__ == "__main__":
    convert_all_documents(INPUT_FOLDER, OUTPUT_FOLDER)


❌ 변환 실패: ./raw_data\(공고 제2024-881호) Burr Saw등 절삭기류 치료재료 비용 산정 가능 행위 목록 공고(고시 2024-258호,12.18. 관련).hwpx → [WinError 2] 지정된 파일을 찾을 수 없습니다
❌ 변환 실패: ./raw_data\(제2025-23호) 요양급여의 적용기준 및 방법에 관한 세부사항 일부개정.hwpx → [WinError 2] 지정된 파일을 찾을 수 없습니다
⚠ HWP 변환 불가, 기록: [별표 7] 진료코드(제20조제7항 관련)(자동차보험진료수가 심사업무처리에 관한 규정) (1).hwp
❌ 변환 실패: ./raw_data\[의과 1기관 4사례] 추간판 장애 등 상병으로 통증조절 위해 입원한 사례 (척추1A).hwpx → [WinError 2] 지정된 파일을 찾을 수 없습니다
⚠ HWP 변환 불가, 기록: 고시 제2025-102호 [일반원칙]항진균제.hwp

📄 변환 불가 파일 기록됨: ./output\미처리_HWP_파일목록.csv

✨ 총 변환 완료: 0개
📌 변환 불가(HWP): 2개
